In [91]:
import random
import math

def cop_kmeans (dataset, k, ml=None, cl=None):
    
    clusters_, centers = initialize_clusters(dataset, k)
    clusters = None

    while not clusters != clusters_:
        clusters = clusters_
        clusters_ = [-1] * len(dataset)
        for i, d in enumerate(dataset):
            found_cluster = False
            indices = closest_clusters(centers, d)
            counter = 0
            while (not found_cluster) and counter < len(indices):
                index = indices[counter]
                if not violate_constraints(i, index, clusters, ml, cl):
                    found_cluster = True
                    clusters_[i] = index
            if not found_cluster:
                return None
            
        centers = compute_centers(clusters_, dataset, k)
        converged = test_convergence (clusters, clusters_)

        
    return clusters, centers

def euclidean_distance(point1, point2):
    return math.sqrt(sum([(i-j)^2 for (i,j) in zip(point1, point2)]))

def closest_clusters(centers, datapoint):
    distances = [euclidean_distance(center, datapoint) for 
                 center in centers]
    return sorted(range(len(distances)), key=lambda x: distances[x])

# TODO : implement carefully, maybe with a look at Davidson's work
def initialize_clusters(dataset, k, ml, cl):
    pass

def violate_constraints(data_index, cluster_index, clusters, ml, cl):
    for (i,j) in ml:
        if i == data_index and clusters[j] != cluster_index:
            return true
        if j == data_index and clusters[i] != cluster_index:
            return true
    for (i,j) in cl:
        if i == data_index and clusters[j] == cluster_index:
            return true
        if j == data_index and clusters[i] == cluster_index:
            return true
    return false

def compute_centers(clusters, dataset, k=None):
    if k == None:
        k = max(clusters) + 1
    dim = len(dataset[0])
    centers = [[]] * k
    for i in range(k):
        center = [0.0] * dim
        count = 1
        for j, c in enumerate(clusters):
            if c == i:
                center += dataset[j]
                count += 1
        centers[i] = [center[d]/float(count) for d in range(dim)]
    return centers
